# Section 0: Import relevant functions

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import random
# Dependencies and Setup
# SQL Alchemy
from sqlalchemy import create_engine

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, PrimaryKeyConstraint

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
import sqlite3

# Section 1: Load, view, and begin to clean the data

Dataset #1

In [3]:
engine = create_engine("sqlite:///Resources/happiness.sqlite")

meta = MetaData()
meta.drop_all(engine)

In [4]:
df1 = pd.read_csv('Resources/2019.csv')

In [5]:
engine.table_names()

['Country', 'Happiness_Score', 'Three_Letter_Codes']

In [6]:
df1.head()

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [7]:
# Identify incomplete rows
df1.count()

Overall rank                    156
Country or region               156
Score                           156
GDP per capita                  156
Social support                  156
Healthy life expectancy         156
Freedom to make life choices    156
Generosity                      156
Perceptions of corruption       156
dtype: int64

In [8]:
print(df1.columns)

Index(['Overall rank', 'Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')


In [9]:
df1 = df1[['Country or region', 'Score', 'GDP per capita',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']]

In [10]:
df1.columns

Index(['Country or region', 'Score', 'GDP per capita', 'Social support',
       'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')

In [11]:
df1.rename(columns={'Country or region':'country_name', 'Score':'score', 'GDP per capita':'gdp_per_capita',
       'Social support':'social_support', 'Healthy life expectancy':'healthy_life_expectancy',
       'Freedom to make life choices':'freedom_to_make_choices', 'Generosity':'generosity',
       'Perceptions of corruption':'perceptions_of_corruption'}, inplace=True)

In [12]:
df1.shape

(156, 8)

In [13]:
# Return non-NAN rows in each column
print(df1.count())

country_name                 156
score                        156
gdp_per_capita               156
social_support               156
healthy_life_expectancy      156
freedom_to_make_choices      156
generosity                   156
perceptions_of_corruption    156
dtype: int64


In [14]:
# Return total number of rows including NAN in the dataframe
df1.size

1248

In [15]:
df1.describe()

,score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_choices,generosity,perceptions_of_corruption
count,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000
mean,5.407096,0.905147,1.208814,0.725244,0.392571,0.184846,0.110603
std,1.113120,0.398389,0.299191,0.242124,0.143289,0.095254,0.094538
min,2.853000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.544500,0.602750,1.055750,0.547750,0.308000,0.108750,0.047000
50%,5.379500,0.960000,1.271500,0.789000,0.417000,0.177500,0.085500
75%,6.184500,1.232500,1.452500,0.881750,0.507250,0.248250,0.141250
max,7.769000,1.684000,1.624000,1.141000,0.631000,0.566000,0.453000


Dataset #2

In [16]:
# Create dataframe number 2

df2 = pd.read_csv('Resources/PI_2019_Data.csv')

In [17]:
df2.head()

,area_name,area_code,area_group,pillar_name,element_name,rank_2009,rank_2010,rank_2011,rank_2012,rank_2013,...,score_2010,score_2011,score_2012,score_2013,score_2014,score_2015,score_2016,score_2017,score_2018,score_2019
0,Afghanistan,AFG,Asia-Pacific,Economic Quality,Dynamism,145,146,148,133,131,...,2.2,2.1,2.9,3.1,2.6,3.4,2.4,2.3,2.3,2.5
1,Afghanistan,AFG,Asia-Pacific,Economic Quality,Fiscal Sustainability,82,64,75,69,55,...,14.3,13.9,14.2,15.0,15.4,15.1,15.5,15.8,16.2,15.8
2,Afghanistan,AFG,Asia-Pacific,Economic Quality,Labour Force Engagement,130,128,127,122,117,...,8.2,8.3,8.6,8.8,8.9,8.9,9.1,9.2,9.3,9.3
3,Afghanistan,AFG,Asia-Pacific,Economic Quality,Macroeconomic Stability,131,65,13,3,7,...,5.8,6.5,7.3,7.0,6.9,5.8,5.3,5.4,5.0,4.9
4,Afghanistan,AFG,Asia-Pacific,Economic Quality,Productivity and Competitiveness,139,152,139,138,143,...,6.7,8.2,8.4,8.0,7.9,8.1,7.9,7.7,7.7,7.7


In [18]:
df2.columns

Index(['area_name', 'area_code', 'area_group', 'pillar_name', 'element_name',
       'rank_2009', 'rank_2010', 'rank_2011', 'rank_2012', 'rank_2013',
       'rank_2014', 'rank_2015', 'rank_2016', 'rank_2017', 'rank_2018',
       'rank_2019', 'score_2009', 'score_2010', 'score_2011', 'score_2012',
       'score_2013', 'score_2014', 'score_2015', 'score_2016', 'score_2017',
       'score_2018', 'score_2019'],
      dtype='object')

In [19]:
# keep only wanted columns
df2 = df2[['area_name','area_code','pillar_name','element_name','rank_2019']]

In [20]:
df2.head()

,area_name,area_code,pillar_name,element_name,rank_2019
0,Afghanistan,AFG,Economic Quality,Dynamism,146
1,Afghanistan,AFG,Economic Quality,Fiscal Sustainability,43
2,Afghanistan,AFG,Economic Quality,Labour Force Engagement,116
3,Afghanistan,AFG,Economic Quality,Macroeconomic Stability,119
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142


In [21]:
df2.rename(columns={'area_name':'country_name',
                    'rank_2019':'rank',
                   'area_code': 'alpha_3_code'}, inplace=True)

In [22]:
df2.head()

,country_name,alpha_3_code,pillar_name,element_name,rank
0,Afghanistan,AFG,Economic Quality,Dynamism,146
1,Afghanistan,AFG,Economic Quality,Fiscal Sustainability,43
2,Afghanistan,AFG,Economic Quality,Labour Force Engagement,116
3,Afghanistan,AFG,Economic Quality,Macroeconomic Stability,119
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142


# Section 2: Filter the data

In [23]:
# find the unique pillars
df2['pillar_name'].unique()

array(['Economic Quality', 'Education', 'Enterprise Conditions',
       'Governance', 'Health', 'Investment Environment',
       'Living Conditions', 'Market Access and Infrastructure',
       'Natural Environment', 'Personal Freedom', 'Safety and Security',
       'Social Capital'], dtype=object)

In [24]:
# find the unique elements
df2['element_name'].unique()

array(['Dynamism', 'Fiscal Sustainability', 'Labour Force Engagement',
       'Macroeconomic Stability', 'Productivity and Competitiveness',
       'Adult Skills', 'Pre-Primary Education', 'Primary Education',
       'Secondary Education', 'Tertiary Education',
       'Burden of Regulation', 'Domestic Market Contestability',
       'Environment for Business Creation', 'Labour Market Flexibility',
       'Executive Constraints', 'Government Effectiveness',
       'Government Integrity', 'Political Accountability',
       'Regulatory Quality', 'Rule of Law', 'Behavioural Risk Factors',
       'Care Systems', 'Longevity', 'Mental Health', 'Physical Health',
       'Preventative Interventions', 'Contract Enforcement',
       'Financing Ecosystem', 'Investor Protection', 'Property Rights',
       'Restrictions on International Investment', 'Basic Services',
       'Connectedness', 'Material Resources', 'Nutrition',
       'Protection from Harm', 'Shelter', 'Border Administration',
       'C

In [25]:
# limit the data to "Economics Quality" and Productivity & competitiveness
df2 = df2.loc[(df2['pillar_name'] == 'Economic Quality') & (df2['element_name'] == 'Productivity and Competitiveness')] 
df2.head()

,country_name,alpha_3_code,pillar_name,element_name,rank
4,Afghanistan,AFG,Economic Quality,Productivity and Competitiveness,142
69,Albania,ALB,Economic Quality,Productivity and Competitiveness,88
134,Algeria,DZA,Economic Quality,Productivity and Competitiveness,118
199,Angola,AGO,Economic Quality,Productivity and Competitiveness,145
264,Argentina,ARG,Economic Quality,Productivity and Competitiveness,74


# Section 3: Merging the data

In [26]:
#merge happiness and covid data sets into a new "combined_data_df"
combined_data_df = pd.merge(df1,df2, on ="country_name", how = "inner")
combined_data_df

,country_name,score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_choices,generosity,perceptions_of_corruption,alpha_3_code,pillar_name,element_name,rank
0,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,FIN,Economic Quality,Productivity and Competitiveness,17
1,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,DNK,Economic Quality,Productivity and Competitiveness,23
2,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,NOR,Economic Quality,Productivity and Competitiveness,36
3,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,ISL,Economic Quality,Productivity and Competitiveness,37
4,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,NLD,Economic Quality,Productivity and Competitiveness,16
...,...,...,...,...,...,...,...,...,...,...,...,...
139,Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411,RWA,Economic Quality,Productivity and Competitiveness,123
140,Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147,TZA,Economic Quality,Productivity and Competitiveness,151
141,Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025,AFG,Economic Quality,Productivity and Competitiveness,142
142,Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035,CAF,Economic Quality,Productivity and Competitiveness,163


# Section 4: Load the data into the database

In [27]:
Base = declarative_base()

In [28]:
# create a class to hold the combined dataframe
class Combined_Data(Base):
    __tablename__ = 'Combined_Country_Data'
    country_name = Column(String, primary_key=True)
    score = Column(Integer)
    gdp_per_capita = Column(Integer)
    social_support = Column(Integer)
    healthy_life_expectancy = Column(Integer)  
    freedom_to_make_life_choices = Column(Integer)
    generosity = Column(Integer)
    perceptions_of_corruption = Column(Integer)
    alpha_3_code = Column(String)
    pillar_name = Column(String)
    element_name = Column(String)
    rank = Column(Integer)

In [29]:
# Connect to metadata
Base.metadata.create_all(engine)
metadata = MetaData(bind=engine)
metadata.reflect()

In [30]:
# Create connection to engine
conn = engine.connect()

In [31]:
# Read Dfs into a list of row objects
Master_data = combined_data_df.to_dict(orient='records')

In [32]:
# Insert data into table using SQLAlchemy
Combined_Country_Table = sqlalchemy.Table('Combined_Country_Data', metadata, PrimaryKeyConstraint('country_name'),
                            autoload=True, extend_existing=True)

In [33]:
Combined_Country_Table

Table('Combined_Country_Data', MetaData(bind=Engine(sqlite:///Resources/happiness.sqlite)), Column('country_name', VARCHAR(), table=<Combined_Country_Data>, primary_key=True, nullable=False), Column('score', INTEGER(), table=<Combined_Country_Data>), Column('gdp_per_capita', INTEGER(), table=<Combined_Country_Data>), Column('social_support', INTEGER(), table=<Combined_Country_Data>), Column('healthy_life_expectancy', INTEGER(), table=<Combined_Country_Data>), Column('freedom_to_make_life_choices', INTEGER(), table=<Combined_Country_Data>), Column('generosity', INTEGER(), table=<Combined_Country_Data>), Column('perceptions_of_corruption', INTEGER(), table=<Combined_Country_Data>), Column('alpha_3_code', VARCHAR(), table=<Combined_Country_Data>), Column('pillar_name', VARCHAR(), table=<Combined_Country_Data>), Column('element_name', VARCHAR(), table=<Combined_Country_Data>), Column('rank', INTEGER(), table=<Combined_Country_Data>), schema=None)

In [34]:
# Execute the inserts of the data
conn.execute(Combined_Country_Table.delete())
conn.execute(Combined_Country_Table.insert(), Master_data)

# Section 5: Validate and query the data

In [36]:
# Query all objects into the database

pd.read_sql_query('select * from Combined_Country_Data', con=engine).head()

,country_name,score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption,alpha_3_code,pillar_name,element_name,rank
0,Finland,7.769,1.340,1.587,0.986,None,0.153,0.393,FIN,Economic Quality,Productivity and Competitiveness,17
1,Denmark,7.600,1.383,1.573,0.996,None,0.252,0.410,DNK,Economic Quality,Productivity and Competitiveness,23
2,Norway,7.554,1.488,1.582,1.028,None,0.271,0.341,NOR,Economic Quality,Productivity and Competitiveness,36
3,Iceland,7.494,1.380,1.624,1.026,None,0.354,0.118,ISL,Economic Quality,Productivity and Competitiveness,37
4,Netherlands,7.488,1.396,1.522,0.999,None,0.322,0.298,NLD,Economic Quality,Productivity and Competitiveness,16
